In [ ]:
# Copyright 2020 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# Install Pipeline SDK - This only needs to be run once in the environment. 
!python3 -m pip install 'kfp>=0.1.31' --user --quiet

# KubeFlow Pipelines - Creating an environment validation pipeline using KFP diagnose_me libraries 
#### Step 0 - Gets all known configurations ( this step does not fail due to errors) 

In [ ]:
def run_diagnose_me():
    """ Prints a dump of gcp environment configurations.

    Raises:
        RuntimeError: If gcp credentials are not configured correctly
    """
 
    # Installing pip3 and kfp, since the base image 'google/cloud-sdk:279.0.0' does not come with pip3 pre-installed.
    import subprocess
    subprocess.run(
      ['curl', 'https://bootstrap.pypa.io/get-pip.py', '-o', 'get-pip.py'],
      capture_output=True)
    subprocess.run(['apt-get', 'install', 'python3-distutils', '--yes'],
                 capture_output=True)
    subprocess.run(['python3', 'get-pip.py'], capture_output=True)
    subprocess.run(['python3', '-m', 'pip', 'install', 'kfp>=0.1.31', '--quiet'],
                 capture_output=True)

    subprocess.run(['kfp', 'diagnose_me'])

#### Step 1 - Validates GCP credentials are configured correctly

In [ ]:
def verify_gcp_credentials():
    """ Verifies if gcp credentials are configured correctly.

    Raises:
        RuntimeError: If gcp credentials are not configured correctly
    """

    # Installing pip3 and kfp, since the base image 'google/cloud-sdk:279.0.0' does not come with pip3 pre-installed.
    import subprocess
    subprocess.run(
      ['curl', 'https://bootstrap.pypa.io/get-pip.py', '-o', 'get-pip.py'],
      capture_output=True)
    subprocess.run(['apt-get', 'install', 'python3-distutils', '--yes'],
                 capture_output=True)
    subprocess.run(['python3', 'get-pip.py'], capture_output=True)
    subprocess.run(['python3', '-m', 'pip', 'install', 'kfp>=0.1.31', '--quiet'],
                 capture_output=True)

    import sys
    from typing import List, Text
    import os
    from kfp.cli.diagnose_me import gcp

    # Get the project ID
    project_config = gcp.get_gcp_configuration(
      gcp.Commands.GET_GCLOUD_DEFAULT, human_readable=False)
    project_id = ''
    if not project_config.has_error:
        project_id = project_config.parsed_output['core']['project']
        print('GCP credentials are configured with access to project: %s ...\n' %
              (project_id))
        print('Following account(s) are active under this pipeline:\n')
        subprocess.run(['gcloud', 'auth', 'list'])
        return

    raise RuntimeError(
      'Project configuration is not accessible with error  %s\n' %
      (project_config.stderr) + 'Follow the instructions at\n' +
      'https://github.com/kubeflow/pipelines/blob/master/manifests/gcp_marketplace/guide.md#gcp-service-account-credentials \n'
      + 'to verify you have configured the required gcp secret.')

#### Step 2 - Print scope configuration for each service account

In [ ]:
def print_scopes():
    """ Prints the scope settings for each instance and service account.

    Raises:
        RuntimeError: If gcp credentials are not configured correctly
    """

    # Installing pip3 and kfp, since the base image 'google/cloud-sdk:279.0.0' does not come with pip3 pre-installed.
    import subprocess
    subprocess.run(
      ['curl', 'https://bootstrap.pypa.io/get-pip.py', '-o', 'get-pip.py'],
      capture_output=True)
    subprocess.run(['apt-get', 'install', 'python3-distutils', '--yes'],
                 capture_output=True)
    subprocess.run(['python3', 'get-pip.py'], capture_output=True)
    subprocess.run(['python3', '-m', 'pip', 'install', 'kfp>=0.1.31', '--quiet'],
                 capture_output=True)

    import sys
    from typing import List, Text 
    import os
    from kfp.cli.diagnose_me import gcp
    import json
    # Get the project ID
    project_config = gcp.get_gcp_configuration(gcp.Commands.GET_GCLOUD_DEFAULT,human_readable=False)
    project_id = ''   
    if not project_config.has_error:
        project_id = project_config.parsed_output['core']['project']
        print('Retrieving service account scope for each instant in project %s ...' % (project_id))
    else: 
        raise RuntimeError('Could not retrieve project ID with error  %s' % (project_config.stderr))
        
    # Get the status of GCP APIs and add the results to a dictionary
    scope_results = gcp.get_gcp_configuration(
        gcp.Commands.GET_SCOPES)
    
    status = []
    
    if scope_results.has_error:
        raise RuntimeError('could not retrieve SCOPE status with error: %s' %(scope_results.stderr))

    for item in scope_results.parsed_output:
        temp = {}
        temp['instance_name'] = item.get('name',None)
        for service_account in item.get('serviceAccounts',[]):
            temp['service_account'] = service_account.get('email',None)
            temp['scopes'] = service_account.get('scopes', None)
        status.append(temp)
        
    # Printing the results in stdout for logging purposes 
    print(json.dumps(status,indent = 4, sort_keys = True))
    
    return

#### Step 3 - Validate if required APIs are enabled in the project

In [ ]:
def verfiy_gcp_apis(target_apis:str):
    """ Verifies if specified APIs are enabled under the gcp project.
    
    
    Args: 
        target_apis: comma separated name of the apis
    
    Raises:
        RuntimeError: If gcp secret is not configured correctly, or service account does not 
        have proper privilege to access the API status. 
    """
    
    # Installing pip3 and kfp, since the base image 'google/cloud-sdk:279.0.0' does not come with pip3 pre-installed.
    import subprocess
    subprocess.run(['curl','https://bootstrap.pypa.io/get-pip.py','-o','get-pip.py'], capture_output=True)
    subprocess.run(['apt-get', 'install', 'python3-distutils','--yes'], capture_output=True)
    subprocess.run(['python3', 'get-pip.py'], capture_output=True)
    subprocess.run(['python3', '-m','pip','install','kfp>=0.1.31', '--quiet'], capture_output=True)
    
    
    import sys
    from typing import List, Text 
    import os
    from kfp.cli.diagnose_me import gcp
    
    # Get the project ID
    project_config = gcp.get_gcp_configuration(gcp.Commands.GET_GCLOUD_DEFAULT,human_readable=False)
    project_id = ''   
    if not project_config.has_error:
        project_id = project_config.parsed_output['core']['project']
        print('Verifying APIs in project %s ...' % (project_id))
    else: 
        raise RuntimeError('Could not retrieve project ID with error  %s' % (project_config.stderr))
        
    # Get the status of GCP APIs and add the results to a dictionary
    api_config_results = gcp.get_gcp_configuration(
        gcp.Commands.GET_APIS)
    
    api_status = {}
    
    if api_config_results.has_error:
        raise RuntimeError('could not retrieve API status with error: %s' %(api_config_results.stderr))
    
    for item in api_config_results.parsed_output:
        api_status[item['config']['name']] =  item['state']
        # printing the results in stdout for logging purposes 
        print('%s %s' % (item['config']['name'], item['state']))
    

    # Check if target apis are enabled 
    api_check_results = True
    error_list = []
    for api in target_apis.replace(' ','').split(','): 
        if 'ENABLED'!= api_status.get(api, 'DISABLED'):
            api_check_results = False
            error_list.append('API \"%s\" is not enabled. To enable this api go to https://pantheon.corp.google.com/apis/library/%s?project=%s' %(api,api,project_id))
            
    if api_check_results:
        return
    else:
        raise RuntimeError('Required APIs are not enabled:\n'+ '\n'.join(error_list))

In [ ]:
import kfp.components as comp

run_diagnose_me_op = comp.func_to_container_op(
    run_diagnose_me, base_image='google/cloud-sdk:279.0.0')

verify_gcp_credentials_op = comp.func_to_container_op(
    verify_gcp_credentials, base_image='google/cloud-sdk:279.0.0')

print_scopes_op = comp.func_to_container_op(
    print_scopes, base_image='google/cloud-sdk:279.0.0')


verify_gcp_apis_op = comp.func_to_container_op(
    verfiy_gcp_apis, base_image='google/cloud-sdk:279.0.0')

In [ ]:
from kfp.gcp import use_gcp_secret
from kfp import dsl

@dsl.pipeline(
    name='Verify KFP Env',
    description="""
    Verifies if env is configured properly by 
    Runs diagnose_me tool in the environment and outputs the results  
    - Verify credentials are set correctly and print out the active service account name
    - Print the current scope for each service account 
    - Verify the specified APIs are enabled in the project. To learn more about
    available APIs go to https://pantheon.corp.google.com/apis/library/."""
)
def verify_gcp_kfp_env(
    target_apis='stackdriver.googleapis.com, storage-api.googleapis.com, '
                'bigquery.googleapis.com, dataflow.googleapis.com'
):
    """A sample pipeline to help verifies KFP environment setup."""
    
    # This pipeline assumes a user-gcp-sa is needed for execution, if no secret is needed,
    # or a different secret is being used following should be updated accordingly. 
    task0 = run_diagnose_me_op().apply(use_gcp_secret('user-gcp-sa'))
    task1 = verify_gcp_credentials_op().apply(use_gcp_secret('user-gcp-sa'))
    task2 = print_scopes_op().apply(use_gcp_secret('user-gcp-sa'))
    task3 = verify_gcp_apis_op(target_apis).apply(use_gcp_secret('user-gcp-sa'))

In [ ]:
import kfp
client = kfp.Client(host='<your-host-name>')

In [ ]:
client.create_run_from_pipeline_func(verify_gcp_kfp_env, arguments={})